<a href="https://colab.research.google.com/github/saimani251/Sports-Analytics-Platform/blob/main/IPL_Bowling_Visualizations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact, Dropdown
import ipywidgets as widgets
from google.colab import output
output.enable_custom_widget_manager()
# Load the dataset
data = pd.read_csv('/content/drive/My Drive/IPL 2008-2023/all_season_bowling_card.csv')
data['economyRate'] = pd.to_numeric(data['economyRate'], errors='coerce')
if data['economyRate'].isnull().any():
    mean_economyRate = data['economyRate'].mean()
    data['economyRate'].fillna(mean_economyRate, inplace=True)
# Function to update and display the plot
def update_plot(selected_player):
    if selected_player == 'All Players':
        filtered_data = data
    else:
        filtered_data = data[data['name'] == selected_player]

    # Recalculate stats
    season_stats_filtered = filtered_data.groupby('season').agg(
        total_wickets=pd.NamedAgg(column='wickets', aggfunc='sum'),
        average_economy=pd.NamedAgg(column='economyRate', aggfunc='mean')
    ).reset_index()

    # Plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=season_stats_filtered['season'], y=season_stats_filtered['total_wickets'],
                             mode='lines+markers', name='Total Wickets'))
    fig.add_trace(go.Scatter(x=season_stats_filtered['season'], y=season_stats_filtered['average_economy'],
                             mode='lines+markers', name='Average Economy Rate', yaxis='y2'))

    # Explicitly setting the figure size
    fig.update_layout(
        title='Season-wise Performance Analysis',
        xaxis_title='Season',
        yaxis_title='Total Wickets',
        yaxis2=dict(title='Average Economy Rate', overlaying='y', side='right'),
        width=1300,  # Set the width of the figure
        height=600   # Set the height of the figure
    )

    fig.show()


# Dropdown for player selection
players_list = ['All Players'] + sorted(data['name'].unique())
player_dropdown = Dropdown(options=players_list)

# Interactive widget
interact(update_plot, selected_player=player_dropdown);


interactive(children=(Dropdown(description='selected_player', options=('All Players', 'A Ashish Reddy', 'A Bad…

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact, Dropdown, widgets
from google.colab import output
output.enable_custom_widget_manager()
import plotly.express as px

color_scale = px.colors.sequential.Viridis
# Load the dataset
data = pd.read_csv('/content/drive/My Drive/IPL 2008-2023/all_season_bowling_card.csv')
data['economyRate'] = pd.to_numeric(data['economyRate'], errors='coerce')
data['overs'] = pd.to_numeric(data['overs'], errors='coerce')

if data['economyRate'].isnull().any():
    mean_economyRate = data['economyRate'].mean()
    data['economyRate'].fillna(mean_economyRate, inplace=True)

# Aggregate data for top bowlers analysis
min_overs = 50  # Minimum overs to be considered for economy rate ranking
bowler_stats = data.groupby('name').agg(
    total_wickets=pd.NamedAgg(column='wickets', aggfunc='sum'),
    total_overs=pd.NamedAgg(column='overs', aggfunc='sum'),
    average_economy=pd.NamedAgg(column='economyRate', aggfunc='mean')
).query('total_overs >= @min_overs')

# Top 10 bowlers by total wickets
top_wicket_takers = bowler_stats.sort_values(by='total_wickets', ascending=False).head(10)

# Top 10 bowlers by best average economy rate
top_economy_bowlers = bowler_stats.sort_values(by='average_economy').head(10)
num_bars = len(top_wicket_takers)
colors_for_bars = px.colors.sample_colorscale(color_scale, samplepoints=num_bars)
# Visualize Top Bowlers by Total Wickets
fig_wickets = go.Figure(data=[go.Bar(
            x=top_wicket_takers['total_wickets'],
            y=top_wicket_takers.index,
            orientation='h',
            marker_color=colors_for_bars)])
fig_wickets.update_layout(title_text='Top 10 Bowlers by Total Wickets',
                          xaxis_title="Total Wickets",
                          yaxis_title="Bowler")
fig_wickets.show()

# Visualize Top Bowlers by Economy Rate
fig_economy = go.Figure(data=[go.Bar(
            x=top_economy_bowlers['average_economy'],
            y=top_economy_bowlers.index,
            orientation='h',
            marker_color=colors_for_bars)])
fig_economy.update_layout(title_text='Top 10 Bowlers by Economy Rate',
                          xaxis_title="Average Economy Rate",
                          yaxis_title="Bowler")
fig_economy.show()

# Dropdown widget and interaction
players_list = ['All Players'] + sorted(data['name'].unique())


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact, Dropdown

# Load the dataset
data = pd.read_csv('/content/drive/My Drive/IPL 2008-2023/all_season_bowling_card.csv')

# Clean and prepare the data
data['economyRate'] = pd.to_numeric(data['economyRate'], errors='coerce')
data.fillna({'economyRate': data['economyRate'].mean()}, inplace=True)

data['venue'] = data['venue'].str.split(',').str[0]  # Extract the first venue name
# Dropdown for player selection
players_list = ['All Players'] + sorted(data['name'].unique())
player_dropdown = Dropdown(options=players_list, description="Select Player:")

# Function to update the bar chart based on the selected player
def update_venue_performance(selected_player):
    if selected_player == "All Players":
        # Calculate average economy rate for all players at each venue
        avg_economy_by_venue = data.groupby('venue')['economyRate'].mean().reset_index()
    else:
        # Filter data for the selected player and calculate average economy rate at each venue
        player_data = data[data['name'] == selected_player]
        avg_economy_by_venue = player_data.groupby('venue')['economyRate'].mean().reset_index()

    # Create the bar chart
    fig = go.Figure(data=[go.Bar(
        x=avg_economy_by_venue['venue'],
        y=avg_economy_by_venue['economyRate'],
        marker_color='indianred'
    )])

    # Update layout
    fig.update_layout(title=f'Average Economy Rate by Venue for {selected_player}',
                      xaxis_title='Venue',
                      xaxis_tickangle=-45,
                      yaxis_title='Average Economy Rate',
                      yaxis=dict(range=[min(avg_economy_by_venue['economyRate'])-1, max(avg_economy_by_venue['economyRate'])+1]),
                      width=1300,
                      height=800)
    fig.show()

# Display widget and bind the update function
interact(update_venue_performance, selected_player=player_dropdown);


interactive(children=(Dropdown(description='Select Player:', options=('All Players', 'A Ashish Reddy', 'A Bado…

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact, Dropdown, VBox, Output
import numpy as np

#Load the Dataseet

data = pd.read_csv('/content/drive/My Drive/IPL 2008-2023/all_season_bowling_card.csv')

# Clean and prepare the data
data['economyRate'] = pd.to_numeric(data['economyRate'], errors='coerce')
data['overs'] = pd.to_numeric(data['overs'], errors='coerce')
data.fillna({'economyRate': data['economyRate'].mean()}, inplace=True)
data['balls_bowled'] = data['overs'].apply(lambda x: int(x) * 6 + (x * 10) % 10)
data['wickets'].replace(0, np.nan, inplace=True)  # Avoid division by zero
data['strike_rate'] = data['balls_bowled'] / data['wickets']
data['home_or_away'] = data.apply(lambda x: 'Home' if x['home_team'] == x['bowling_team'] else 'Away', axis=1)

# Sample player selection dropdown
players_list = ['All Players'] + sorted(data['name'].unique())
player_dropdown = Dropdown(options=players_list, description="Select Player:")

# Output widgets for the visualizations
economy_output = Output()
strike_output = Output()
wickets_output = Output()

def create_visualizations(selected_player):
    # Filter data for the selected player or all players
    filtered_data = data if selected_player == "All Players" else data[data['name'] == selected_player]

    # Generate visualizations
    # Economy Rate Visualization
    with economy_output:
        economy_output.clear_output()
        # Prepare data for economy rate visualization
        economy_data = filtered_data.groupby(['season', 'home_or_away'])['economyRate'].mean().reset_index()
        fig_economy = go.Figure()
        for home_away in ['Home', 'Away']:
            # Filter and plot each subset
            subset = economy_data[economy_data['home_or_away'] == home_away]
            fig_economy.add_trace(go.Bar(x=subset['season'], y=subset['economyRate'], name=home_away))
        fig_economy.update_layout(title='Economy Rate by Season (Home vs. Away)', barmode='group',width=1300, height=600)
        fig_economy.show()

    # Strike Rate Visualization
    with strike_output:
        strike_output.clear_output()
        # Prepare data for strike rate visualization
        strike_data = filtered_data.groupby(['season', 'home_or_away'])['strike_rate'].mean().reset_index()
        fig_strike = go.Figure()
        for home_away in ['Home', 'Away']:
            # Filter and plot each subset
            subset = strike_data[strike_data['home_or_away'] == home_away]
            fig_strike.add_trace(go.Bar(x=subset['season'], y=subset['strike_rate'], name=home_away))
        fig_strike.update_layout(title='Strike Rate by Season (Home vs. Away)', barmode='group',width=1300, height=600)
        fig_strike.show()

    # Wickets Visualization
    with wickets_output:
        wickets_output.clear_output()
        # Prepare data for wickets visualization
        wickets_data = filtered_data.groupby(['season', 'home_or_away'])['wickets'].sum().reset_index()
        fig_wickets = go.Figure()
        for home_away in ['Home', 'Away']:
            # Filter and plot each subset
            subset = wickets_data[wickets_data['home_or_away'] == home_away]
            fig_wickets.add_trace(go.Bar(x=subset['season'], y=subset['wickets'], name=home_away))
        fig_wickets.update_layout(title='Wickets by Season (Home vs. Away)', barmode='group',width=1300, height=600)
        fig_wickets.show()

# Bind the player selection dropdown to the visualization function
interact(create_visualizations, selected_player=player_dropdown)

# Organize and display the Output widgets in a vertical layout
display(VBox([economy_output, strike_output, wickets_output]))


interactive(children=(Dropdown(description='Select Player:', options=('All Players', 'A Ashish Reddy', 'A Bado…

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interact, Dropdown, Output
from google.colab import output
output.enable_custom_widget_manager()
# Load the dataset
data = pd.read_csv('/content/drive/My Drive/IPL 2008-2023/all_season_bowling_card.csv')

# Prepare the data
data['dots'] = pd.to_numeric(data['dots'], errors='coerce')
data['foursConceded'] = pd.to_numeric(data['foursConceded'], errors='coerce')
data['sixesConceded'] = pd.to_numeric(data['sixesConceded'], errors='coerce')
data['total_boundaries'] = data['foursConceded'] + data['sixesConceded']

# Dropdown for player selection
players_list = ['All Players'] + sorted(data['name'].unique())
player_dropdown = Dropdown(options=players_list, description="Select Player:")

# Create an output widget to display the plot
plot_output = Output()

@interact(selected_player=player_dropdown)
def create_scatter_plot(selected_player):
    # Clear the previous plot
    plot_output.clear_output()

    # Filter data for the selected player or all players
    filtered_data = data if selected_player == "All Players" else data[data['name'] == selected_player]

    # Group data by season
    season_stats = filtered_data.groupby('season').agg(
        total_dots=pd.NamedAgg(column='dots', aggfunc='sum'),
        total_boundaries=pd.NamedAgg(column='total_boundaries', aggfunc='sum')
    ).reset_index()

    # Create the scatter plot
    fig = go.Figure(data=go.Scatter(
        x=season_stats['total_dots'],
        y=season_stats['total_boundaries'],
        mode='markers',
        marker=dict(size=15, color='LightSkyBlue', line=dict(width=2, color='DarkSlateGrey')),
        hovertext=season_stats['season'],
        hoverinfo='text+x+y',
    ))

    # Update plot layout
    fig.update_layout(
        title=f"Dot Balls vs. Boundaries for {selected_player}",
        xaxis_title="Total Dot Balls",
        yaxis_title="Total Boundaries",
        width=1300,
        height=600
    )

    # Display the plot in the output widget
    with plot_output:
        fig.show()

# Display the plot output widget
plot_output


interactive(children=(Dropdown(description='Select Player:', options=('All Players', 'A Ashish Reddy', 'A Bado…

Output()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import interactive, Dropdown, Layout

# Load the dataset
data = pd.read_csv('/content/drive/My Drive/IPL 2008-2023/all_season_bowling_card.csv')

# Prepare the data
data['dots'] = pd.to_numeric(data['dots'], errors='coerce')

# Create dropdown for player selection
players_list = ['All Players'] + sorted(data['name'].unique())
player_dropdown = Dropdown(options=players_list, description="Select Player:", layout=Layout(width='300px'))

# Visualization function
def create_extras_visualization(selected_player):
    filtered_data = data if selected_player == "All Players" else data[data['name'] == selected_player]
    extras_by_season = filtered_data.groupby('season').agg(
        wides=('wides', 'sum'),
        no_balls=('noballs', 'sum'),

    ).reset_index()

    fig = go.Figure()
    fig.add_trace(go.Bar(x=extras_by_season['season'], y=extras_by_season['wides'], name='Wides'))
    fig.add_trace(go.Bar(x=extras_by_season['season'], y=extras_by_season['no_balls'], name='No Balls'))

    fig.update_layout(
        barmode='stack',
        title='Extras Analysis by Season',
        xaxis_title='Season',
        yaxis_title='Number of Extras',
        width=1300,
        height=600
    )

    fig.show()

# Use interactive to create a widget for the visualization function
interactive_widget = interactive(create_extras_visualization, selected_player=player_dropdown)

# Display the interactive widget
display(interactive_widget)


interactive(children=(Dropdown(description='Select Player:', layout=Layout(width='300px'), options=('All Playe…

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, Dropdown
from google.colab import output
output.enable_custom_widget_manager()

# Load the dataset
data = pd.read_csv('/content/drive/My Drive/IPL 2008-2023/match_data2.csv')

# Group data to calculate sums of b_balls and b_outs
grouped_data = data.groupby(['season', 'Phase', 'bowler']).agg(
    total_b_balls=pd.NamedAgg(column='b_balls', aggfunc='sum'),
    total_b_outs=pd.NamedAgg(column='b_outs', aggfunc='sum')
).reset_index()


# Handling division by zero by replacing 0 outs with NaN
grouped_data['strike_rate'] = np.where(grouped_data['total_b_outs'] > 0,
                                       grouped_data['total_b_balls'] / grouped_data['total_b_outs'],
                                       np.nan)

# Function to plot the chart for strike rate
def plot_strike_rate(bowler):
    if bowler != 'All Players':
        filtered_data = grouped_data[grouped_data['bowler'] == bowler]
    else:
        filtered_data = grouped_data.copy()

    fig = go.Figure()
    for phase in filtered_data['Phase'].unique():
        phase_data = filtered_data[filtered_data['Phase'] == phase]
        seasons = phase_data['season'].unique()
        strike_rates = [phase_data[phase_data['season'] == season]['strike_rate'].mean() for season in seasons]

        fig.add_trace(go.Bar(
            x=seasons,
            y=strike_rates,
            name=phase,
            text=phase,
        ))

    # Update the layout
    fig.update_layout(
        barmode='stack',
        title_text=f'Strike Rate by Season and Phase for {bowler}',
        xaxis_title='Season',
        yaxis_title='Strike Rate',
        legend_title='Phase',
        width=1300,
        height=600
    )

    fig.show()

# Create a dropdown and use interact to update the chart
bowlers = ['All Players'] + sorted(grouped_data['bowler'].unique().tolist())
dropdown = Dropdown(options=bowlers)

# Display interactive widget
_=interact(plot_strike_rate, bowler=dropdown)


interactive(children=(Dropdown(description='bowler', options=('All Players', 'A Ashish Reddy', 'A Badoni', 'A …

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, Dropdown
from google.colab import output
output.enable_custom_widget_manager()

# Load the dataset
data = pd.read_csv('/content/drive/My Drive/IPL 2008-2023/match_data2.csv')

# Group data to calculate sums of runs_off_bat and b_balls
grouped_data = data.groupby(['season', 'Phase', 'bowler']).agg(
    total_runs=pd.NamedAgg(column='runs_off_bat', aggfunc='sum'),
    total_b_balls=pd.NamedAgg(column='b_balls', aggfunc='sum')
).reset_index()

# Handling division by zero by replacing 0 b_balls with NaN
grouped_data['economy_rate'] = np.where(grouped_data['total_b_balls'] > 0,
                                        grouped_data['total_runs'] / (grouped_data['total_b_balls'] / 6.0),
                                        np.nan)

# Function to plot the chart for economy rate
def plot_economy_rate(bowler):
    if bowler != 'All Players':
        filtered_data = grouped_data[grouped_data['bowler'] == bowler]
    else:
        filtered_data = grouped_data.copy()

    fig = go.Figure()
    for phase in filtered_data['Phase'].unique():
        phase_data = filtered_data[filtered_data['Phase'] == phase]
        seasons = phase_data['season'].unique()
        economy_rates = [phase_data[phase_data['season'] == season]['economy_rate'].mean() for season in seasons]

        fig.add_trace(go.Bar(
            x=seasons,
            y=economy_rates,
            name=phase,
            text=phase,
        ))

    # Update the layout
    fig.update_layout(
        barmode='stack',
        title_text=f'Economy Rate by Season and Phase for {bowler}',
        xaxis_title='Season',
        yaxis_title='Economy Rate',
        legend_title='Phase',
        width=1300,
        height=600
    )

    fig.show()

# Create a dropdown and use interact to update the chart
bowlers = ['All Players'] + sorted(grouped_data['bowler'].unique().tolist())
dropdown = Dropdown(options=bowlers)

# Display interactive widget
_=interact(plot_economy_rate, bowler=dropdown)


interactive(children=(Dropdown(description='bowler', options=('All Players', 'A Ashish Reddy', 'A Badoni', 'A …